In [2]:
'''This notebook is used to control the Standa motorized polarizers and the Pylon camera. It also
	measures the power reaching the sample and makes some preliminar plots.
'''


import os
import time
import numpy as np
import scipy
from scipy import interpolate
from matplotlib.pyplot import *
import matplotlib.patches as patches
import scipy.optimize as optimize
from pyvisa import * 
import time
import pickle 
ion()

#print(os.getcwd())
###loading Pylon camera
os.chdir("C:/Users/Administrateur/Documents/Scripts_Python/Drivers_instruments/Camera_Pylon/")
import sys

sys.path.append(os.getcwd())
import Pylon_init as pyl

##import motorized polarizers
os.chdir("C:/Users/Administrateur/Documents/ximc/scripts/GotThePower/")
import StandaPolar_init_v32bits as std

lib = std.lib
device_id1 = std.device_id1
# device_id2 = std.device_id2

####################################################################################
dir_0 = r"C:\Users\Administrateur\Desktop\Sotos\\"
dir_run="testdir\\"
try:
	os.mkdir(dir_0+dir_run)
except:
	print("exists already")

4
5
6
7
8
You are now connected to <modules.picam_types.PicamCameraID object at 0x13F08A00>
The sensor temperature is -120.0 C
The target temperature is -120.0 C
The exposure time is 1.0 ms
Current ROIs [(0, 1340, 1, 65, 25, 25)]
Library loaded
Library version: 2.10.5
Device count: 1
Connected device 1:  b'xi-com:\\\\.\\COM10'
Connected device 2:  None
Connected device 3:  None
device_id1:  1
device_id2:  -1
device_id3:  -1

Standa motorized polarizer ready to be used. Remind that the two first arguments for fct calling are always 'lib' and 'device_id'. Do not forget to close the device when you are done by calling the function close(lib, device_id).
exists already


In [3]:
#Setting up the powermeter. If not working, you may want to try turning off and on the powermeter

from datetime import datetime
from ctypes import cdll,c_long, c_ulong, c_uint32,byref,create_string_buffer,c_bool,c_char_p,c_int,c_int16,c_double, sizeof, c_voidp
from TLPMX import TLPMX
import time

from TLPMX import TLPM_DEFAULT_CHANNEL

# Find connected power meter devices.
tlPM = TLPMX()
deviceCount = c_uint32()
tlPM.findRsrc(byref(deviceCount))

print("Number of found devices: " + str(deviceCount.value))
print("")

resourceName = create_string_buffer(1024)

for i in range(0, deviceCount.value):
    tlPM.getRsrcName(c_int(i), resourceName)
    print("Resource name of device", i, ":", c_char_p(resourceName.raw).value)
print("")
tlPM.close()

# Connect to last device.
tlPM = TLPMX()
tlPM.open(resourceName, c_bool(True), c_bool(True))

message = create_string_buffer(1024)
tlPM.getCalibrationMsg(message,TLPM_DEFAULT_CHANNEL)
print("Connected to device", i)
print("Last calibration date: ",c_char_p(message.raw).value)
print("")

time.sleep(2)

# Set wavelength in nm.
wavelength = c_double(532.0)
tlPM.setWavelength(wavelength,TLPM_DEFAULT_CHANNEL)

# Enable auto-range mode.
# 0 -> auto-range disabled
# 1 -> auto-range enabled
tlPM.setPowerAutoRange(c_int16(1),TLPM_DEFAULT_CHANNEL)

# Set power unit to Watt.
# 0 -> Watt
# 1 -> dBm
tlPM.setPowerUnit(c_int16(0),TLPM_DEFAULT_CHANNEL)

Number of found devices: 1

Resource name of device 0 : b'USB0::0x1313::0x8078::P0037864::INSTR'

Connected to device 0
Last calibration date:  b'07-Oct-2022'



0

In [5]:
#Function to measure power before/after taking spectra
def measure_mean_power(time_span, steps): #time span in seconds
    measurements = np.zeros(steps)
    for i in range(steps):
        measured_power = c_double()
        tlPM.measPower(byref(measured_power),TLPM_DEFAULT_CHANNEL)
        measurements[i] = measured_power.value
        time.sleep(time_span/steps)
    return np.mean(measurements)


def delta_points(spectrum):

    #bkg = np.mean(spectrum[0:20])
    #spectrum = spectrum - bkg 

    amax = np.argmax(spectrum)
    max_value = np.mean(spectrum[amax-5:amax+6])


    normalized_spectrum = spectrum/max_value
    temp_spec = np.abs(normalized_spectrum-0.5)

    amin1 = np.argmin(temp_spec[0:amax])
    amin2 = np.argmin(temp_spec[amax:])

    delta_arg = amin2+amax-amin1
    #plt.plot(normalized_spectrum)
    #plt.plot([amin1,amin2+amax],[0.5,0.5],'o')
    # plt.show()
    return delta_arg


In [6]:
%matplotlib qt
current_date = time.strftime('%Y_%m_%d')
dir_0 = r"C:\Users\Administrateur\Desktop\Sotos\\"
dir_run=f"Measurements{current_date}\\"
try:
	os.mkdir(dir_0+dir_run)
except:
	print("There's already a folder for today")



There's already a folder for today


In [7]:
#Automatic power calibration

##Find the power for a given half-waveplate angle
angle_offset_correction = -9
min_angle = 9
max_angle = 55
angles_calib = np.linspace(min_angle, max_angle, 10) #These angles are for power calibration

powers_calib = np.zeros(len(angles_calib)) 

std.move(lib, device_id1, min_angle+angle_offset_correction)

for ii in range(len(angles_calib)):
    time.sleep(3)
    std.move(lib, device_id1, angles_calib[ii]+angle_offset_correction)
    powers_calib[ii] = measure_mean_power(5,10)
    time.sleep(3)

std.move(lib, device_id1, min_angle+angle_offset_correction)

actual_power_coef = 11*0.5*1e6 # 11 is how many time the measured power crosses the BS, 0.5is how much of it crosses the 50/50 (uW)

interpolation_ND0 = interpolate.interp1d(powers_calib*actual_power_coef, angles_calib)

print(f'Power range ({min(powers_calib*actual_power_coef):.2f}, {max(powers_calib*actual_power_coef):.2f}) uW')
print('Complete !!!')



Going to 0 deg

Going to 0 steps, 0 microsteps
Result: 0
Wait...

Done

Going to 0.0 deg

Going to 0 steps, 0 microsteps
Result: 0
Wait...

Done

Going to 5.111111111111111 deg

Going to 408 steps, 228 microsteps
Result: 0
Wait...

Done

Going to 10.222222222222221 deg

Going to 817 steps, 199 microsteps
Result: 0
Wait...

Done

Going to 15.333333333333332 deg

Going to 1226 steps, 171 microsteps
Result: 0
Wait...

Done

Going to 20.444444444444443 deg

Going to 1635 steps, 142 microsteps
Result: 0
Wait...

Done

Going to 25.555555555555557 deg

Going to 2044 steps, 114 microsteps
Result: 0
Wait...

Done

Going to 30.666666666666664 deg

Going to 2453 steps, 85 microsteps
Result: 0
Wait...

Done

Going to 35.77777777777777 deg

Going to 2862 steps, 57 microsteps
Result: 0
Wait...

Done

Going to 40.888888888888886 deg

Going to 3271 steps, 28 microsteps
Result: 0
Wait...

Done

Going to 46.0 deg

Going to 3680 steps, 0 microsteps
Result: 0
Wait...

Done

Going to 0 deg

Going to 0 ste

In [24]:
#Wavelenght calibration. This cell is probably useless now, it's better take a calibration spectrum with Lightfield
'''
current_time = time.strftime("%H%M%S")
grating = 150 #Grooves mm-1
time_acquisition = 0.5     #seconds  
center = 800 #nm
calib_filename = f'Calibration_{grating}_gr_{time_acquisition}_s_{current_time}_center_{center}_nm'

pyl.acquire_and_save(dir_0+dir_run, calib_filename, time_acquisition*1000)

#Uncomment to check calibration

calib_neon = np.loadtxt(dir_0+dir_run+calib_filename+'.dat')
plot(calib_neon)
title('Neon calibration')

show()
'''

'\ncurrent_time = time.strftime("%H%M%S")\ngrating = 150 #Grooves mm-1\ntime_acquisition = 0.5     #seconds  \ncenter = 800 #nm\ncalib_filename = f\'Calibration_{grating}_gr_{time_acquisition}_s_{current_time}_center_{center}_nm\'\n\npyl.acquire_and_save(dir_0+dir_run, calib_filename, time_acquisition*1000)\n\n#Uncomment to check calibration\n\ncalib_neon = np.loadtxt(dir_0+dir_run+calib_filename+\'.dat\')\nplot(calib_neon)\ntitle(\'Neon calibration\')\n\nshow()\n'

In [26]:
#Background measurements

current_time = current_time = time.strftime("%H%M%S")
time_acquisition = np.array([1, 2, 5, 10])     #seconds
for ii in range(len(time_acquisition)): 
    background_filename = f'Background_{time_acquisition[ii]}_s_{current_time}'
    pyl.acquire_and_save(dir_0+dir_run, background_filename, time_acquisition[ii]*1000)
    plot(np.loadtxt(dir_0+dir_run+background_filename+'.dat'), label = f'{time_acquisition[ii]} s')
legend()
show()


Acquisition of a 1.0 s spectrum
Spectrum saved in C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\ as Background_1_s_105736
Acquisition of a 2.0 s spectrum
Spectrum saved in C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\ as Background_2_s_105736
Acquisition of a 5.0 s spectrum
Spectrum saved in C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\ as Background_5_s_105736
Acquisition of a 10.0 s spectrum
Spectrum saved in C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\ as Background_10_s_105736


In [47]:
#A list where each spectrum will be stored
spectra = []
min_power = 100 #Must be more than powers_calib[0]
max_power = 3200 #Must be less than powers_calib[-1]
steps = 10

powers = np.linspace(min_power, max_power, steps)
angles = interpolation_ND0(powers)

current_time = time.strftime("%H%M%S")
dir_data = f'{current_time}\\'

reversibility = True
acquisition_time = 1000 #miliseconds

powers_reversibility = np.concatenate([powers,powers[::-1][:]])

#Creation of a folder
try:
    os.mkdir(dir_0+dir_run+dir_data)
except:
    print('Folder name for this data exists already, wait a little')

#Comments
with open(dir_0+dir_run+dir_data+'comment.txt','w') as comment:
    user_input = input('Any comment?')
    comment.write(user_input+'\n'+'acquisition time: '+str(acquisition_time)+'ms'+'\n'+'Max power at the sample:'+str(max_power)+'uW')

#Power measurements file
if reversibility == True:
    measured_powers = np.zeros(len(powers_reversibility))
else:
    measured_powers = np.zeros(len(powers))
#Going forwards
for ii in range(len(angles)):

    std.move(lib, device_id1, angles[ii] + angle_offset_correction)
    filename = f"{angles[ii]:.2f}_degrees"
    
    time.sleep(3)
    measured_powers[ii] = measure_mean_power(1,5)
    pyl.acquire_and_save(dir_0+dir_run+dir_data,filename,acquisition_time)
    time.sleep(3)
    
    f = open(dir_0+dir_run+dir_data+filename+".dat","r")
    tmp = f.readlines()
    f.close()
    
    spectrum=[]
    for k in range(len(tmp)):
        tmp_ = list(map(float,tmp[k].split("\n'")))
        spectrum.append(tmp_)
    
    spectra.append(spectrum)



#Routine to go backwards, comment if not needed
if reversibility ==True:
    
    reversed_angles = angles[::-1]
    for jj in range(len(reversed_angles)):
        std.move(lib, device_id1, reversed_angles[jj] + angle_offset_correction)
        
        filename = f"{reversed_angles[jj]:.2f}_degrees_back"
        
        time.sleep(3)
        measured_powers[len(powers)+jj] = measure_mean_power(1,5)
        pyl.acquire_and_save(dir_0+dir_run+dir_data,filename,acquisition_time)
        time.sleep(3)

        f = open(dir_0+dir_run+dir_data+filename+".dat","r")
        tmp = f.readlines()
        f.close()
        
        spectrum=[]
        for k in range(len(tmp)):
            tmp_ = list(map(float,tmp[k].split("\n'")))
            spectrum.append(tmp_)
        
        spectra.append(spectrum)
#Save data and go back to the minimal power

spectra = np.array(spectra).squeeze()
np.savetxt(dir_0+dir_run+dir_data+'spectra.txt', spectra.T, delimiter='\t', fmt='%.6f')
np.savetxt(dir_0+dir_run+dir_data+'measured_powers.txt', measured_powers, delimiter='\t', fmt='%.12f')
std.move(lib, device_id1, min_angle+angle_offset_correction)

print('Complete!!!')



Going to 3.4887506651874087 deg

Going to 279 steps, 26 microsteps
Result: 0
Wait...

Done
Acquisition of a 1.0 s spectrum
Spectrum saved in C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\ as 12.49_degrees

Going to 10.643397698936983 deg

Going to 851 steps, 121 microsteps
Result: 0
Wait...

Done
Acquisition of a 1.0 s spectrum
Spectrum saved in C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\ as 19.64_degrees

Going to 14.561567424791157 deg

Going to 1164 steps, 237 microsteps
Result: 0
Wait...

Done
Acquisition of a 1.0 s spectrum
Spectrum saved in C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\ as 23.56_degrees

Going to 17.89683323730879 deg

Going to 1431 steps, 191 microsteps
Result: 0
Wait...

Done
Acquisition of a 1.0 s spectrum
Spectrum saved in C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\ as 26.90_degrees

Going to 21.05579738040692 deg

Going to 1684 steps, 119 microsteps
Result: 0
W

In [48]:
#Cascade plot

import pickle

def save_interactive_fig(fig, pkl_filename='saved_figure.pkl', py_filename='load_saved_figure.py'):
    """
    Saves a Matplotlib figure as a pickle file and generates a Python script to reload and display it.
    
    Parameters:
        fig (matplotlib.figure.Figure): The Matplotlib figure to save.
        pkl_filename (str): Filename for the saved pickle file (default: 'saved_figure.pkl').
        py_filename (str): Filename for the generated Python script (default: 'load_saved_figure.py').
    """
    # Save the figure as a pickle file
    with open(pkl_filename, 'wb') as f:
        pickle.dump(fig, f)
    print(f"Figure saved as '{pkl_filename}'.")
    
    # Create the Python script to reload and display the figure
    loader_script = f"""
import pickle
import matplotlib.pyplot as plt

# Load the figure from the file
with open(r'{pkl_filename}', 'rb') as f:
    fig = pickle.load(f)

# Show the figure
fig.show()
plt.show()  # Keep the plot open until the user closes it
"""
    with open(py_filename, 'w') as f:
        f.write(loader_script)
    print(f"Loader script saved as '{py_filename}'.")



background_1s = np.loadtxt(dir_0+dir_run+'Background_1_s_105736.dat')
reversibility = True
norm =  matplotlib.pyplot.Normalize(min(powers), max(powers))


fig, ax = subplots()

if reversibility ==True:
    powers_reversibility = np.concatenate([powers,powers[::-1][:]])
    fig.set_size_inches(10, 6)
    for ii in range(len(spectra)):
        
        if ii < len(powers):
            cmap = matplotlib.cm.Reds 
        else:
            cmap = matplotlib.cm.Blues
        plot((spectra[ii]-background_1s)/(acquisition_time*1e-3), label = f'{measured_powers[ii]*actual_power_coef:.2f} uW', color = cmap(norm(powers_reversibility[ii])))
        
else:
    fig.set_size_inches(10, 6)
    for ii in range(len(spectra)):
        
        plot((spectra[ii]-background_1s)/(acquisition_time*1e-3), label = f'{measured_powers[ii]*actual_power_coef:.2f} uW')


ylabel("Intensity(counts/s)")
xlabel("Pixels")
title("Power dependant measurement.")
legend(loc = (0.65,0.4), ncol =2)
#tight_layout()
save_interactive_fig(fig, dir_0+dir_run+dir_data+r'Cascade.pkl', dir_0+dir_run+dir_data+r'Cascade.pyx')
savefig(dir_0+dir_run+dir_data+'Cascade.pdf')

show()

Figure saved as 'C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\Cascade.pkl'.
Loader script saved as 'C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\Cascade.pyx'.


In [49]:
#Width change
fig, ax = subplots()

widths = np.zeros(len(spectra))
for ii in range(len(spectra)):
    widths[ii] = delta_points(spectra[ii])

plot(measured_powers[0:len(powers)]*actual_power_coef, widths[0:len(powers)], 'ro', label = 'Going up')
plot(measured_powers[len(powers):]*actual_power_coef, widths[len(powers):], 'bo', label = 'Going down')


title('Width variation')
xlabel('Power (uW)')
ylabel('Width (pixels)')

legend()
save_interactive_fig(fig, dir_0+dir_run+dir_data+r'Widths.pkl', dir_0+dir_run+dir_data+r'Widths.pyx')

savefig(dir_0+dir_run+dir_data+'Widths.pdf', dpi=300)
show()


Figure saved as 'C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\Widths.pkl'.
Loader script saved as 'C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\Widths.pyx'.


In [53]:
#Normalized Cascade plot
fig, ax = subplots()

background_1s = np.loadtxt(dir_0+dir_run+'Background_1_s_105736.dat')
reversibility = True
norm =  matplotlib.pyplot.Normalize(min(powers), max(powers))

if reversibility ==True:
    powers_reversibility = np.concatenate([powers,powers[::-1][:]])
    fig.set_size_inches(10, 6)
    for ii in range(len(spectra)):
        if ii < len(powers):
            cmap = matplotlib.cm.Reds 
        else:
            cmap = matplotlib.cm.Blues
        plot((spectra[ii]-background_1s)/ max(spectra[ii]-background_1s), label = f'{measured_powers[ii]*actual_power_coef:.2f} uW', color = cmap(norm(powers_reversibility[ii])))
        
else:
    fig.set_size_inches(10, 6)
    for ii in range(len(spectra)):
        plot((spectra[ii]-background_1s)/ max(spectra[ii]-background_1s)/(acquisition_time*1e-3), label = f'{measured_powers[ii]*actual_power_coef:.2f} uW')


ylabel("Intensity (normalized to max)")
xlabel("Pixels")
title("Power dependant measurement.")
legend(loc = (0.65,0.4), ncol =2)
#tight_layout()
save_interactive_fig(fig, dir_0+dir_run+dir_data+r'NormalizedCascade.pkl', dir_0+dir_run+dir_data+r'NormalizedCascade.pyx')
savefig(dir_0+dir_run+dir_data+'NormalizedCascade.pdf', dpi=300)
show()


Figure saved as 'C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\NormalizedCascade.pkl'.
Loader script saved as 'C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\NormalizedCascade.pyx'.


In [54]:
#Intensity and position of the maximals. Also integrated intensity
maximals = []
positions = []
integrated_intensity = []

for ii in range(len(spectra)):

    maximals.append(max(spectra[ii] - background_1s))
    positions.append(np.argmax(spectra[ii] - background_1s))

    integrated_intensity.append(scipy.integrate.simpson(np.array(spectra[ii] - background_1s).flatten()))



In [55]:
#Only for reversibility measurements
fig, ax = subplots()

reversibility = True
if reversibility == True:
    powers_reversibility = np.concatenate([powers,powers[::-1][:]])
    plot(measured_powers[0:len(powers)] * actual_power_coef, integrated_intensity[0:len(powers)], 'ro', label = 'Going up')
    plot(measured_powers[len(powers):] * actual_power_coef, integrated_intensity[len(powers):], 'bo', label = 'Going down')
    title('Reversibility test')
else: 
    plot(powers/(acquisition_time*1e-3), integrated_intensity)
    title('Integrated intensity as a function of optical excitation power')

xlabel("Power (uW)")
ylabel("Integrated intensity (counts/s)")

legend()
save_interactive_fig(fig, dir_0+dir_run+dir_data+r'Integrated_intensity.pkl', dir_0+dir_run+dir_data+r'Integrated_intensity.pyx')

savefig(dir_0+dir_run+dir_data+'Integrated_intensity.pdf')
show()

Figure saved as 'C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\Integrated_intensity.pkl'.
Loader script saved as 'C:\Users\Administrateur\Desktop\Sotos\\Measurements2025_01_17\153741\Integrated_intensity.pyx'.
